Network for arteficial data

In [1]:
import pandas as pd
import numpy as np
import torch
from scipy.integrate import odeint

In [2]:
#load data and set up device and initial conditions
data = pd.read_csv('/workspaces/bio-pinn/Wiebke (Arteficial Data)/Data_noisy.csv')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
y0 = np.array([1., 0., 0., 0.])
print(data)


     time     ConcA     ConcB     ConcC     ConcD
0     0.0  0.994669 -0.005747 -0.013655  0.003871
1     0.1  0.892689  0.085837  0.095478  0.008688
2     0.2  0.824135  0.194037  0.150377  0.043714
3     0.3  0.752378  0.265149  0.201116  0.100012
4     0.4  0.678789  0.334336  0.190981  0.105275
..    ...       ...       ...       ...       ...
96    9.6  0.153484  0.869480  0.338554  0.512678
97    9.7  0.134096  0.858041  0.353139  0.519071
98    9.8  0.140395  0.868077  0.326654  0.501598
99    9.9  0.150394  0.855664  0.354416  0.504462
100  10.0  0.151180  0.846118  0.355460  0.502241

[101 rows x 5 columns]


In [3]:
#define pinn
class Net(torch.nn.Module):
    
    def __init__(self, k):
        super().__init__()
        self.actf = torch.tanh
        self.f1 = torch.nn.Linear(1, 100)
        self.f2 = torch.nn.Linear(100, 100)
        self.f3 = torch.nn.Linear(100, 100)
        self.f4 = torch.nn.Linear(100, 4)
        
        self.k1 = torch.nn.Parameter(torch.tensor(k[0], device=device))
        self.k2 = torch.nn.Parameter(torch.tensor(k[1], device=device))
        self.k3 = torch.nn.Parameter(torch.tensor(k[2], device=device))
        self.k4 = torch.nn.Parameter(torch.tensor(k[3], device=device))

    def forward(self, x):
        x = self.actf(self.f1(x))
        x = self.actf(self.f2(x))
        x = self.actf(self.f3(x))
        x = self.f4(x)
        return x.squeeze()

from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self, in_tensor, out_tensor):
        self.inp = in_tensor
        self.out = out_tensor

    def __len__(self):
        return len(self.inp)

    def __getitem__(self, idx):
        return self.inp[idx], self.out[idx]

In [4]:
from torch.autograd import grad
#define physics-informed loss function
def phys_loss(t, C, model):
    dCdt = torch.autograd.grad(
        C, t,
        grad_outputs=torch.ones_like(C),
        create_graph=True
    )[0]

    A, B, Cc, D = C[:,0], C[:,1], C[:,2], C[:,3]

    fA = -model.k1 * A
    fB = model.k1 * A - model.k2 * B
    fC = model.k2 * B - model.k3 * Cc
    fD = model.k3 * Cc - model.k4 * D

    rhs = torch.stack([fA, fB, fC, fD], dim=1)
    return torch.mean((dCdt - rhs)**2)

In [5]:
#set up model, optimizer, and training data
k = [7., 10., .1, 3.]       #estimations for ks, the closer to the real values the lower the error
from torch.optim import Adam
model_pinn = Net(k).to(device)

epochs = 1000
optimizer_pinn = Adam(model_pinn.parameters(), lr=0.1)
loss_fcn = torch.nn.MSELoss()
df=data[['ConcA','ConcB','ConcC','ConcD']]

train_in = torch.tensor(data['time'].values, dtype=torch.float32, requires_grad=True).view(-1, 1).to(device)
train_out = torch.tensor(df.values, dtype=torch.float32).to(device)
time=torch.tensor(data['time'].values, dtype=torch.float32)


for epoch in range(epochs):
    model_pinn.train()  # Ensure model is in training mode

    optimizer_pinn.zero_grad()
    pred = model_pinn(train_in)
    base_loss = loss_fcn(pred, train_out)
    phys_loss_value = phys_loss(train_in, pred, model_pinn)
    total_loss = base_loss + phys_loss_value
    total_loss.backward()
    optimizer_pinn.step()

    # Print losses for every epoch
    print(f'Epoch {epoch+1}/{epochs} | Loss: {total_loss.item():.4f} = {base_loss.item():.4f} (Base) + {phys_loss_value.item():.4f} (Physics)')






Epoch 1/1000 | Loss: 1.0553 = 0.2260 (Base) + 0.8293 (Physics)
Epoch 2/1000 | Loss: 142.4505 = 1.2994 (Base) + 141.1511 (Physics)
Epoch 3/1000 | Loss: 10907.0117 = 94.2165 (Base) + 10812.7949 (Physics)
Epoch 4/1000 | Loss: 1440.9690 = 17.3873 (Base) + 1423.5817 (Physics)
Epoch 5/1000 | Loss: 820.4992 = 13.1722 (Base) + 807.3270 (Physics)
Epoch 6/1000 | Loss: 2926.6558 = 28.4970 (Base) + 2898.1587 (Physics)
Epoch 7/1000 | Loss: 187.2207 = 23.3817 (Base) + 163.8390 (Physics)
Epoch 8/1000 | Loss: 43.4094 = 26.8004 (Base) + 16.6091 (Physics)
Epoch 9/1000 | Loss: 149.8918 = 18.9277 (Base) + 130.9641 (Physics)
Epoch 10/1000 | Loss: 208.5655 = 7.0126 (Base) + 201.5529 (Physics)
Epoch 11/1000 | Loss: 118.3599 = 1.2218 (Base) + 117.1381 (Physics)
Epoch 12/1000 | Loss: 31.3583 = 6.9730 (Base) + 24.3853 (Physics)
Epoch 13/1000 | Loss: 51.8552 = 18.9733 (Base) + 32.8819 (Physics)
Epoch 14/1000 | Loss: 119.1282 = 24.8866 (Base) + 94.2417 (Physics)
Epoch 15/1000 | Loss: 117.8961 = 20.1480 (Base) + 9

Final comments:

One could add a loop for optimizing the k-value estimations to optimize the final results <br>
One colud also add a break-off condition for the epochs <br>
I also tried a lot off stuff with the experimental data in a different repo, because i did found it difficult to understand which data related to which and what all the abbriviations ment
In the end i got all the notebooks in the original repo given by the instructor to work but that was about it.